In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install --upgrade transformers

In [ ]:
%%capture
!pip install unsloth

from unsloth import FastVisionModel
import torch
import os
import re
from tqdm import tqdm
from PIL import Image
from transformers import TextStreamer

In [ ]:
model, tokenizer = FastVisionModel.from_pretrained(
    "unsloth/Qwen2.5-VL-32B-Instruct-bnb-4bit",
    # "unsloth/Llama-3.2-11B-Vision-Instruct-bnb-4bit",
    load_in_4bit = True, # Use 4bit to reduce memory use. False for 16bit LoRA.
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for long context
    device_map="balanced",  # Distribute across both T4 GPUs
    low_cpu_mem_usage=True,
    trust_remote_code=True,
    # Aggressive memory limits for T4x2
    max_memory={
        0: "13GB",  # GPU 0 - leave some buffer
        1: "13GB",  # GPU 1 - leave some buffer
        "cpu": "20GB"
    },
    offload_folder="./offload_temp",
    offload_state_dict=True,
)

In [ ]:
total = sum(p.numel() for p in model.parameters())
print(f"Total params: {total/1e9:.2f} billion")

In [ ]:
# Enable for inference
FastVisionModel.for_inference(model)
model.eval()  # set to eval mode

def solve_geometry_problem_with_image(image_path, question, choices):
    """
    Solve geometry problem using Qwen2.5-VL Model with step-by-step reasoning

    Args:
        image_path (str): Path to the geometry problem image
        question (str): Question to solve
        choices (str): Multiple choice options (A, B, C, D)

    Returns:
        dict: Contains 'content'
    """
    # Load the image
    try:
        image = Image.open(image_path).convert('RGB')
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        return {'content': 'Error loading image'}
    
    # Enhanced prompt to force choice selection with vision
    prompt = f"""You are an AI mathematician solving geometric problems through visual analysis and logical deduction.

GEOMETRY PROBLEM IMAGE:
The image shows a geometric figure with various shapes, lines, angles, and measurements. Analyze this image carefully to understand the geometric relationships and constraints.

QUESTION:
{question}

ANSWER CHOICES:
{choices}

SOLUTION STEPS:
1. Carefully examine the geometric figure in the image and identify all the given information
2. Understand what measurements, angles, or relationships are shown
3. Identify what needs to be found based on the question
4. Apply relevant geometric theorems, properties, and formulas step by step
5. Show all your calculations clearly
6. Compare your calculated result with the given answer choices
7. Even if your calculated answer doesn't exactly match any choice, select the closest one

⚠️ CRITICAL OUTPUT FORMAT REQUIREMENT ⚠️
YOU MUST END YOUR RESPONSE WITH EXACTLY ONE OF THESE FOUR LINES:
Final Answer: A
Final Answer: B
Final Answer: C
Final Answer: D

❌ ABSOLUTELY FORBIDDEN - DO NOT USE:
- "The final answer is $\\boxed{{14}}$"
- "The final answer is $\\boxed{{A}}$"
- "$\\boxed{{A}}$"
- "\\boxed{{A}}"
- "(A)"
- "A is correct."
- "Final Answer: The answer is A"
- Any LaTeX formatting
- Any mathematical notation
- Any additional text after the letter

✅ REQUIRED FORMAT EXAMPLES:
If you determine the answer is choice A: "Final Answer: A"
If you determine the answer is choice B: "Final Answer: B"
If you determine the answer is choice C: "Final Answer: C"
If you determine the answer is choice D: "Final Answer: D"

IMPORTANT: Your response must end with exactly "Final Answer: [SINGLE LETTER]" - nothing else on that line. Do not include any boxed notation, LaTeX, or mathematical formatting in your final line.

Begin your analysis now and remember to end with the exact required format."""

    # Create messages for Qwen2.5-VL
    messages = [
        {
            "role": "user", 
            "content": [
                {"type": "image"},
                {"type": "text", "text": prompt}
            ]
        }
    ]

    # Apply chat template
    input_text = tokenizer.apply_chat_template(
        messages, 
        add_generation_prompt=True
    )
    
    # Prepare inputs with image
    inputs = tokenizer(
        image,
        input_text,
        add_special_tokens=False,
        return_tensors="pt",
    ).to("cuda")

    with torch.no_grad():
        # Generate response with Qwen2.5-VL parameters
        outputs = model.generate(
            **inputs,
            max_new_tokens=3000,
            temperature=0.8,
            min_p=0.1,
            use_cache=True,
            do_sample=True,
        )
    
    torch.cuda.empty_cache()

    # Decode only the generated tokens (excluding input)
    generated_tokens = outputs[:, inputs.input_ids.shape[-1]:]
    content = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0].strip()

    return {
        'content': content
    }

In [ ]:
def extract_answer_letter(content):
    """
    Enhanced function to extract an answer letter from a model's output.
    It includes patterns for plain text, markdown, and LaTeX formats to ensure
    the answer is captured reliably.
    Args:
        content (str): The model's output content.
    Returns:
        str: The extracted answer letter (A, B, C, or D), or an empty string if not found.
    """
    # Enhanced list of regex patterns to try, in order of preference
    patterns = [
        # LaTeX box patterns - FIXED PATTERNS
        r"\\boxed\{([A-D])\}",                # Handles '\boxed{A}' or '$\boxed{A}$'
        r"\$\\boxed\{([A-D])\}\$",            # Handles '$\boxed{A}$'
        r"\\boxed\{\\text\{([A-D])\}\}",      # Handles '\boxed{\text{A}}'
        r"\$\\boxed\{\\text\{([A-D])\}\}\$",  # Handles '$\boxed{\text{A}}$'
        r"The final answer is \$\\boxed\{([A-D])\}\$",  # 'The final answer is $\boxed{A}$'
        r"The final answer is \\boxed\{([A-D])\}",      # 'The final answer is \boxed{A}'
        r"The final answer is \$\\boxed\{(\d+)\}\$",    # Extract from numeric boxed answers
        
        # Standard patterns
        r"Final Answer:\s*([A-D])\b",        # Final Answer: A
        r"Final Answer:\s*\*\*([A-D])\*\*",  # Final Answer: **A**
        r"Answer:\s*([A-D])\b",              # Answer: A
        r"Answer:\s*\*\*([A-D])\*\*",        # Answer: **A**
        r"Answer:\s*\*([A-D])\*",            # Answer: *A*
        r"Answer:\s*_([A-D])_",              # Answer: _A_
        r"Answer:\s*\(([A-D])\)",            # Answer: (A)
        r"Answer:\s*([A-D])\.",              # Answer: A.
        
        # Sentence-based patterns
        r"The answer is\s*([A-D])\b",        # The answer is A
        r"The correct answer is\s*([A-D])\b", # The correct answer is A
        r"\b([A-D])\s*is the correct",       # A is the correct
        
        # Choice/option patterns
        r"choice\s*([A-D])\b",               # choice A
        r"option\s*([A-D])\b",               # option A
        r"select\s*([A-D])\b",               # select A
        r"choose\s*([A-D])\b",               # choose A
        
        # Concluding word patterns
        r"Therefore,?\s*([A-D])\b",          # Therefore, A
        r"Thus,?\s*([A-D])\b",               # Thus, A
        r"Hence,?\s*([A-D])\b",              # Hence, A
    ]
    
    # Try each pattern in the defined order
    for pattern in patterns:
        match = re.search(pattern, content, re.IGNORECASE)
        if match:
            captured = match.group(1).upper()
            # Handle numeric answers by mapping to choices if needed
            if captured.isdigit():
                # You might need to implement logic here to map numbers to letters
                # based on your specific answer choices
                continue
            return captured
    
    # Special handling for boxed numeric answers like "The final answer is $\boxed{14}$"
    # Try to match the numeric value with your answer choices
    numeric_boxed = re.search(r"\\boxed\{([0-9.]+)\}", content)
    if numeric_boxed:
        numeric_value = numeric_boxed.group(1)
        # You would need to compare this with your actual answer choices
        # and return the corresponding letter
        # For now, we'll continue to other patterns
        pass
    
    # If no specific pattern matches, look for isolated letters near the end
    lines = content.strip().split('\n')
    for line in reversed(lines[-10:]):  # Check the last 10 lines
        line = line.strip()
        if line in ['A', 'B', 'C', 'D']:
            return line
        # Check if a line contains only one of the possible answer letters
        letters_found = re.findall(r'\b([A-D])\b', line)
        if len(letters_found) == 1:
            return letters_found[0].upper()
    
    # As a last resort, find any occurrence of A, B, C, or D in the content
    all_letters = re.findall(r'\b([A-D])\b', content)
    if all_letters:
        # Return the last one found, as it's most likely the final answer
        return all_letters[-1].upper()
    
    return ""

In [ ]:
def validate_and_retry_if_needed(image_path, question, choices, max_retries=3):
    """
    Try to get a valid answer letter, with retries if needed.
    """
    for attempt in range(max_retries):
        result = solve_geometry_problem_with_image(image_path, question, choices)
        content = result['content']
        answer_letter = extract_answer_letter(content)
        
        if answer_letter in ['A', 'B', 'C', 'D']:
            return content, answer_letter
        
        print(f"Attempt {attempt + 1} failed to extract valid answer letter")
    
    # If all attempts fail, try one more time with a very direct prompt
    try:
        image = Image.open(image_path).convert('RGB')
        direct_prompt = f"""Look at this geometry problem image and answer the question.

QUESTION: {question}
ANSWER CHOICES: {choices}

⚠️ CRITICAL OUTPUT FORMAT REQUIREMENT ⚠️
YOU MUST END YOUR RESPONSE WITH EXACTLY ONE OF THESE FOUR LINES:
Final Answer: A
Final Answer: B
Final Answer: C
Final Answer: D

❌ ABSOLUTELY FORBIDDEN - DO NOT USE:
- "The final answer is $\\boxed{{14}}$"
- "The final answer is $\\boxed{{A}}$"
- "$\\boxed{{A}}$"
- "\\boxed{{A}}"
- "(A)"
- "A is correct."
- "Final Answer: The answer is A"
- Any LaTeX formatting
- Any mathematical notation
- Any additional text after the letter

✅ REQUIRED FORMAT EXAMPLES:
If you determine the answer is choice A: "Final Answer: A"
If you determine the answer is choice B: "Final Answer: B"
If you determine the answer is choice C: "Final Answer: C"
If you determine the answer is choice D: "Final Answer: D"

IMPORTANT: Your response must end with exactly "Final Answer: [SINGLE LETTER]" - nothing else on that line. Do not include any boxed notation, LaTeX, or mathematical formatting in your final line.

Begin your analysis now and remember to end with the exact required format.
"""
        
        messages = [
            {
                "role": "user", 
                "content": [
                    {"type": "image"},
                    {"type": "text", "text": direct_prompt}
                ]
            }
        ]
        
        input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
        inputs = tokenizer(image, input_text, add_special_tokens=False, return_tensors="pt").to("cuda")
        
        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=3000, temperature=0.8, min_p=0.1, use_cache=True, do_sample=True)
        
        generated_tokens = outputs[:, inputs.input_ids.shape[-1]:]
        final_content = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0].strip()
        final_letter = extract_answer_letter(final_content)
        
        return final_content, final_letter if final_letter in ['A', 'B', 'C', 'D'] else 'A'  # Default to A if still fails
    
    except Exception as e:
        print(f"Error in final retry: {e}")
        return "Error occurred", 'A'

In [ ]:
if __name__ == "__main__":
    # Input directories
    image_dir = "image"  # Changed from predicates_dir to image_dir
    questions_dir = "/question"
    choices_dir = "choices"
    
    # Output directories
    reasoning_output_dir = "/kaggle/working/reasoning_output"
    answer_literal_dir = "/kaggle/working/answer_literal_qwen2_5_vl"  # Updated name
    os.makedirs(reasoning_output_dir, exist_ok=True)
    os.makedirs(answer_literal_dir, exist_ok=True)
    
    # Iterate over problem numbers
    for num_str in tqdm(range(2401, 3002)):        
        # Paths to input files
        image_path = os.path.join(image_dir, f"{num_str}.png")  # Changed to .png extension
        ques_path = os.path.join(questions_dir, f"{num_str}.txt")
        choice_path = os.path.join(choices_dir, f"{num_str}.txt")
        
        # Check if all required files exist
        if not all(os.path.exists(path) for path in [image_path, ques_path, choice_path]):
            print(f"Skipping problem {num_str}: Missing input files")
            continue
        
        # Read inputs
        try:
            with open(ques_path, "r") as f:
                question = f.read().strip()
            with open(choice_path, "r") as f:
                choices = f.read().strip()
        except Exception as e:
            print(f"Error reading files for problem {num_str}: {e}")
            continue
        
        # Solve with validation and retry mechanism
        simple_content, answer_letter = validate_and_retry_if_needed(image_path, question, choices)
        
        # Build the reasoning file content
        reasoning_lines = []
        reasoning_lines.append("=" * 100)
        reasoning_lines.append("PROBLEM DETAILS:")
        reasoning_lines.append("=" * 100)
        reasoning_lines.append(f"IMAGE: {image_path}")  # Changed from PREDICATES to IMAGE
        reasoning_lines.append("")
        reasoning_lines.append(f"QUESTION:\n{question}")
        reasoning_lines.append("")
        reasoning_lines.append(f"CHOICES:\n{choices}")
        reasoning_lines.append("")
        
        reasoning_lines.append("=" * 100)
        reasoning_lines.append("QWEN2.5-VL MODEL RESPONSE:")  # Updated model name
        reasoning_lines.append("=" * 100)
        reasoning_lines.append(simple_content)
        reasoning_lines.append("")
        reasoning_lines.append("=" * 100)
        reasoning_lines.append(f"EXTRACTED ANSWER: {answer_letter}")
        reasoning_lines.append("=" * 100)
        
        reasoning_output = "\n".join(reasoning_lines)
        
        # Write reasoning output to file
        reasoning_out_path = os.path.join(reasoning_output_dir, f"{num_str}.txt")
        with open(reasoning_out_path, "w") as f:
            f.write(reasoning_output)
        
        # Validate answer letter
        if not answer_letter or answer_letter not in ['A', 'B', 'C', 'D']:
            print(f"Warning: Invalid answer letter '{answer_letter}' for problem {num_str}")
            print(f"Content: {simple_content[:200]}...")
            # Force a default answer rather than empty
            answer_letter = 'A'  # Default fallback
        
        # Write just the answer letter to a separate file
        letter_out_path = os.path.join(answer_literal_dir, f"{num_str}.txt")
        with open(letter_out_path, "w") as f:
            f.write(answer_letter)
        
        print(f"Problem {num_str}: Answer = {answer_letter}")